# Preference Tuning for Summarization using Synthetic Data

**⏱️ Time to complete**: \<TODO\>

Preference tuning is a powerful tool that can optimize LLMs towards complex preferences that can not easily captured through supervised fine-tuning. However, manually annotating preferences between model outputs using human raters can be extremely time-consuming and expensive. Instead, synthetic preference data can be generated by scoring responses with large foundation models, allowing for much cheaper and scalable data collection!

Here we'll go through an end-to-end example for preference tuning of an open-source language model with synthetic data, covering data preprocessing, fine-tuning and evaluation. We will focus on the task of summarization for the [CNN/DailyMail](https://huggingface.co/datasets/abisee/cnn_dailymail) dataset. 

This notebook is based on the following blog post: `TODO`.

In our blog post, we use `Meta-Llama-3.1-70B-Instruct` to judge the summaries and perform full parameter fine-tuning for DPO. However, this requires A100 GPUs, so in this notebook we set the default configs to leverage `Meta-Llama-3.1-8B-Instruct` and LoRA fine-tuning using A10G GPUs. We also have additional configs for A100s if you wish to replicate the results from the blog post.

# Table of Contents
1. [Data Preprocessing](#step-1-data-preprocessing): In this section we cover how we can prepare preference data for the summarization task using an LLM-as-a-judge. 
2. [DPO Finetuning](#step-2-fine-tuning): This section will cover how you can fine-tune an open source model on the preference data on the Anyscale platform.
3. [Evaluation](#step-3-evaluation): The section will lay down a blue-print for evaluation and compare performance to that of closed source models like OpenAI's GPT-4.

First, let's make the necessary imports

In [7]:
import os
import yaml
import datasets
import openai

import ray.data

import pprint
import textwrap

os.environ["PYTHONPATH"] = f"{os.environ.get('PYTHONPATH', '')}:src"

# Step 1: Synthetic Data Generation

First, let's inspect the training dataset and look at an example. 

In [ ]:
hf_ds = datasets.load_dataset("abisee/cnn_dailymail", '3.0.0', split="train").shuffle(seed=21)
# extract a subset of 20000 articles
hf_ds_subset =  hf_ds.select(range(20000))

ray_ds = ray.data.from_huggingface(hf_ds_subset)
raw_example = ray_ds.take(1)[0]

In [5]:
pprint.pprint(raw_example, width=80)

{'article': 'Scam: Lisa Harrison, 34, promised customers low currency rates on '
            'US dollars and special deals . A wedding planner who stole '
            "£80,000 from couples in a bid to satisfy an 'out-of-control' "
            'online gambling addiction has been jailed. Lisa Harrison, 34, '
            'began taking money from her clients in summer 2013 by enticing '
            'them with low currency rates on US dollars and flight upgrades. '
            'She took money from 19 couples who had entrusted their savings to '
            'her after being promised the wedding of their dreams. It is '
            'understood that the company she worked for, iPlan New York, '
            'specialised in weddings in New York City. Her website '
            "iplannewyork.com, which has been taken down, said: 'iPlan New "
            'York was set up to create and style the perfect tailor made '
            "wedding for couples travelling to New York to get married! 'We "
     

Now, we need to get preference data for pairs of summaries generated from the same article. Traditionally, this would involve generating summaries using the base model you wish to fine-tune and asking human annotators to provide a rating for each sample. In this example, we will employ a _synthetic_ summary scoring method using an LLM as a judge. We score the correctness of a summary using the following metrics:

**Summary Scoring Metrics**
1. Multiple choice Q&A accuracy:
    - Given the original text, we use an LLM judge to generate 5 multiple choice questions about the text.
    - We then ask the LLM judge to answer the questions using only the summary, and record the number of questions correctly answered.
2. Word count: We simply count the number of words in the summary.

This allows us to construct a simple preference function between two summaries:

**Preference Function**
1. If both summary responses attain ≥3 multiple choice questions correct, we will prefer the shorter response. We do not care about Q&A accuracy beyond 3 correct answers, since the summary should not contain all information from the text.
2. Otherwise, we select the response that leads to more correctly answered multiple choice questions.

To generate the training data, we will generate 10 summaries from each article using the model we wish to fine-tune. Then, we will randomly sample pairs of summaries and use our preference function to annotate the preference between them.

For this example, we will use `Mistral-7B-Instruct-v0.1` as the base model to fine-tune and `Llama-3.1-70B-Instruct` as a judge.

Combining all this together, our data pre-processing pipeline is going to look as follows: 

![preprocessing](./assets/preprocessing.png?1)



TODO
\<We have the relevant preprocessing code in `utils/generate_questions.py` and `utils/generate_summaries_and_scores.py`. You can run data generation as an Anyscale job with configs/generate_questions_job.yaml and configs/generate_summaries_job.yaml.\>

\<After preprocessing, here's an example for the Q&A generated by Llama 70B and here's an example for the summaries generated by Mistral 7B Instruct \>


\<We sample chosen and rejected messages from the summaries based on the Q&A Accuracy score. We use a threshold of 3/5 for classifying examples as 'chosen' and 'rejected'. Here's an example training dataset sample for the DPO model\>

### Generate Multiple Choice Questions from Articles

First, we will generate the multiple choice questions and answers for each article using `Llama-3.1-8B-Instruct` (or `70B` if you have A100/H100s). Leveraging vLLM and Ray, we can very easily scale this generation process across multiple GPUs.

The following command will run the [src/scripts/generate_questions.py](./src/scripts/generate_questions.py) script, which generates the questions and answers and saves them in `.parquet` files.

In [ ]:
!anyscale job submit -f configs/generate_questions_job.yaml

Output
(anyscale +1.4s) Submitting job with config JobConfig(name='preference-tuning-summarization-question-generation', image_uri='localhost:5555/anyscale/endpoints_aica:0.5.0-6402', compute_config=None, env_vars=None, py_modules=None, cloud=None, project=None, ray_version=None, job_queue_config=None).
(anyscale +3.6s) Using workspace runtime dependencies env vars: {'WANDB_API_KEY': 'cbc4aed2de2d9c9acb21324a3297b85b7299479b'}.
(anyscale +3.6s) Uploading local dir '.' to cloud storage.
(anyscale +5.0s) Including workspace-managed pip dependencies.
(anyscale +5.6s) Job 'preference-tuning-summarization-question-generation' submitted, ID: 'prodjob_sdaruzx8uu3c2bu3x5dn6gpf77'.
(anyscale +5.6s) View the job in the UI: https://console.anyscale.com/jobs/prodjob_sdaruzx8uu3c2bu3x5dn6gpf77
(anyscale +5.6s) Use `--wait` to wait for the job to run and stream logs.


At the end of the job, you should see the remote path to the folder with Q&A. Make sure to make note to use it for the next steps! 

In [ ]:
# Replace this with the link to the output folder from the previous job
qa_folder = f"s3://air-example-data/preference-tuning-summarization-example/qa_generation/qa_annotations_full_train/"
qa_ds = ray.data.read_parquet(qa_folder)
# The dataset is small, we can materalize it
example_rows = qa_ds.materialize().take(3)

In [39]:
from src.utils.models import DataSchema

for row in example_rows:
    print("TEXT:")
    print(textwrap.fill(row[DataSchema.ARTICLE], width=80))
    print("QUESTIONS:")
    print(textwrap.fill(row[DataSchema.MCQ_QUESTIONS], width=80))
    print("ANSWERS:")
    print(textwrap.fill(str(row[DataSchema.GROUND_TRUTH_MCQ_ANSWERS]), width=80))
    pprint.pprint("=" * 100, width=80)

TEXT:
(RollingStone.com) -- Jennifer Lawrence, the 20-year-old Oscar nominee for Best
Actress, is sitting in a fancy Manhattan hotel sipping tea and feeling a little
out of place. See, she grew up in Louisville, Kentucky, where her dad owned a
construction company and her mom ran a summer camp. They had land and horses.
She loved to fish. She was a total tomboy: field hockey, softball, basketball on
an all-boys team. ("I was so dykey.") One of her nicknames was Nitro. She lives
in Los Angeles now, but "little redneck things still come out." Like what? "I'm
attracted to my brother. Stuff like that." 10 Best Movies of 2010 . At 14, she
decided she wanted to be an actress and dragged her mom to New York for
auditions. The people at Reese's Peanut Butter Cups told her she was the best
they'd ever seen. Her mom told her they were lying. (Her mom didn't like showbiz
much.) She auditioned for the role of Bella in "Twilight," which would have been
perfect if Bella were a badass, but since she'

### Generate Summaries + Scores

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the `TODO` script, which takes in the folder of questions and generates the results to a new folder of `.parquet` files.

In [25]:
!anyscale job submit -f configs/generate_summaries_train_job.yaml

Output
(anyscale +1.1s) Submitting job with config JobConfig(name='preference-tuning-summarization-question-generation', image_uri='localhost:5555/anyscale/endpoints_aica:0.5.0-6402', compute_config=None, env_vars=None, py_modules=None, cloud=None, project=None, ray_version=None, job_queue_config=None).
(anyscale +3.5s) Using workspace runtime dependencies env vars: {'WANDB_API_KEY': 'cbc4aed2de2d9c9acb21324a3297b85b7299479b'}.
(anyscale +3.5s) Uploading local dir '.' to cloud storage.
(anyscale +4.5s) Including workspace-managed pip dependencies.
(anyscale +5.1s) Job 'preference-tuning-summarization-question-generation' submitted, ID: 'prodjob_8m2iu1lcd44s2e7q95rcrxvzzx'.
(anyscale +5.1s) View the job in the UI: https://console.anyscale.com/jobs/prodjob_8m2iu1lcd44s2e7q95rcrxvzzx
(anyscale +5.1s) Use `--wait` to wait for the job to run and stream logs.


In [ ]:
# replace with the link to the generated summaries
summary_folder = f"s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/train/" 
summary_ds = ray.data.read_parquet(summary_folder)
example_rows = summary_ds.take(1)

In [41]:
from src.utils.models import DataSchema

for row in example_rows:
    print("TEXT:")
    print(textwrap.fill(row[DataSchema.ARTICLE], width=80))
    print("QUESTIONS:")
    print(textwrap.fill(row[DataSchema.MCQ_QUESTIONS], width=80))
    print("MODEL GENERATED SUMMARY:")
    print(textwrap.fill(row[DataSchema.SUMMARY_GENERATION_RAW_OUTPUT], width=80))
    print("ANSWERS:")
    print(textwrap.fill(str(row[DataSchema.GROUND_TRUTH_MCQ_ANSWERS]), width=80))
    print("JUDGE ANSWERS FROM SUMMARY:")
    print(textwrap.fill(str(row[DataSchema.JUDGE_MCQ_ANSWERS]), width=80))
    pprint.pprint("=" * 100, width=80)

TEXT:
A 43-year-old mother has died in a house fire in Mount Helen, near Ballarat.
Police originally feared the woman and her son and daughter, both 21, had died
in the blaze but they have been safely located, the Ballarat Courier reported. A
neighbour said she heard a 'whooshing' noise like a firecracker and a loud
explosion before the fire started at about 1.30am on Friday. Two women and a man
feared dead in house fire at Mount Helen, Victoria . Next-door neighbour
Margaret Bell witnessed the explosion and called 000. 'I got up to grab a drink
of water and I went back to bed,' Ms Bell told Daily Mail Australia. 'Then I
heard a noise, you know how when a firecracker goes off it makes that whooshing
sort of a noise, then that stopped and I heard a big explosion.' Ms Bell got out
of bed and looked out her window to see the house in flames so she called the
fire service. She added that the family had moved to Mount Helen from Ballarat
just over one week ago. 'I'd spoken to the lady and h

### Generate Preference Tuning Data

Next, we will generate 10 summaries for each article in the training set and score them with our Q&A judging setup. 

The following command will run the `TODO` script, which takes in the folder of summaries and outputs `.jsonl` files for training and validation.

In [ ]:
!python src/scripts/generate_dpo_data.py configs/training_data_generation/mistral_8b.yaml

In [ ]:
# Replace with the link to your validation file
validation_file = f"s3://air-example-data/preference-tuning-summarization-example/dpo_training_data/valid.jsonl"

valid_ds = ray.data.read_json(validation_file)
example_rows = valid_ds.take(1)

In [57]:
for row in example_rows:
    print("PROMPT:")
    print(textwrap.fill(row['chosen'][0]['content'], width=80))
    print("\nCHOSEN RESPONSE: ")
    print(textwrap.fill(row['chosen'][1]['content'], width=80))
    print("\nREJECTED RESPONSE: ")
    print(textwrap.fill(row['rejected'][1]['content'], width=80))


PROMPT:
Given the following text, create a very short summary that is at most 2
sentences.  Text: By . Tamara Cohen, Political Reporter . PUBLISHED: . 18:32
EST, 27 January 2013 . | . UPDATED: . 08:48 EST, 28 January 2013 . Deputy Prime
Minister Nick Clegg and his wife Miriam are determined to keep the education of
their 11-year-old son 'out of politics' Nick Clegg yesterday defended the
possibility he may send his children to private schools as it emerged he and his
wife Miriam have not even visited their local state school. He said the
education of his 11-year-old son Antonio, who starts secondary school this year,
should not be used as 'a political football' and that the couple would do
'what's best' for their children although he was braced for criticism. Last week
the Liberal Democrat leader told listeners to his radio show he would send his
son to a private school if he failed to find a place in a good comprehensive,
saying he would use the state system 'if it works out', but tha

# Step 2: Fine-tuning

Now that we have the pre-processed dataset, we are ready to fine-tune `Mistral-7B-Instruct-v0.1` using DPO. On Anyscale, we've created an easy-to-use interface to do preference-tuning using `DPO`. We leverage Ray to overlap reference model log-probability calculation with model training to improve GPU utilization. Most implementations compute log probabilities synchronously with model training,

![hf model](assets/hf_dpo.png)

While our implementation using Ray is asynchronous:  


![assistant model](assets/anyscale_dpo.png)

Further, our use of Ray Data also implies that the compute configuration for the reference model can be completely decoupled with the policy model. For example, reference model calculation can run on a different node (with configurable number of GPUs, etc) with zero code changes needed. 


To get started with DPO training, we provide the config for DPO in [configs/mistral_dpo_summarization.yaml](configs/mistral_dpo_summarization.yaml) . 

In [10]:
!cat configs/mistral_dpo_summarization.yaml

model_id: mistralai/Mistral-7B-Instruct-v0.1
# Example summarization dataset with 10k examples for training with an average of 2.2k tokens per sample
train_path: s3://air-example-data/preference-tuning-summarization/train.jsonl
valid_path: s3://air-example-data/preference-tuning-summarization/valid.jsonl
task: "preference_tuning"
context_length: 4096
# For DPO, it is recommended to set a high `num_data_blocks_per_device` to not bottleneck the logp processor.
# We recommend not going beyond 20 so as to not spawn too many Ray actors. 
num_data_blocks_per_device: 16
num_devices: 6 # <--- runs training on 6 GPUs
train_batch_size_per_device: 2
eval_batch_size_per_device: 2
learning_rate: 5e-6
num_epochs: 3
no_gradient_checkpoint: False
output_dir: /mnt/local_storage/
deepspeed:
  config_path: deepspeed_configs/zero_3.json
worker_resources:
  accelerator_type:A10G: 1
flash_attention_2: True
padding: "longest"
preference_tuning_config:
  beta: 0.01
  logprob_processor_scaling_config:
    cust

In [ ]:
!llmforge anyscale finetune end-to-end-examples/fine-tune-preference/configs/mistral_dpo_summarization.yaml

# Step 3: Evaluation

Let's evaluate our trained model. Here we'll use two baselines: (1) the base model before finetuning (reference model in DPO) and (2) GPT-4o.

## Evaluation strategy

Our evaluation strategy involves the same Q&A scoring system as used while generating the preference data. 

<p align="center">
  <img src="./assets/eval.png?" alt="Evaluation" width=800>
</p>

We evaluate the baseline model and the trained DPO model on the test set. 

## Obtain summaries on the test set
First, we'll need to obtain the summaries (and scores) for both the models on the given test set. 

>  **_NOTE:_**  The configs will by default use H100s for model scoring and summary generation. for faster processing. Feel free to change the accelerator type in [configs/summary_generation](configs/summary_generation/) but note that the speed would be much slower. 

For the baseline model, you can simply run the below command:

In [ ]:
!anyscale job submit -f configs/generate_summaries_eval_baseline_job.yaml

For the fine-tuned DPO model, we provide a dummy config in [configs/summary_generation/mistral_finetuned_eval.yaml](configs/summary_generation/mistral_finetuned_eval.yaml). If you used the default training config provided, the model would be trained using LoRA and you should have a path to the LoRA weights. Make sure to download the weights locally (using `aws` or `gcloud` CLI depending on the remote path). Enter the local path to the weights in the config. (Make sure that the weights are in the same directory as the current notebook to be included in the job).  

In [4]:
!cat configs/summary_generation/mistral_finetuned_eval.yaml

mode: eval
input_folder: s3://air-example-data/preference-tuning-summarization-example/qa_generation/qa_annotations_full_test
model_inference_config:
  model_id_or_path: mistralai/Mistral-7B-Instruct-v0.1
  adapter_id_or_path: <path to lora model> # <--- Add the local path to your lora weights here
  temperature: 0
  top_p: 0.95
  scaling_config:
    batch_size: 128
    concurrency: 2
    num_gpus: 1
    custom_resources:
      accelerator_type:H100: 1
num_generations: 1
judge_inference_config:
  model_id_or_path: meta-llama/Meta-Llama-3.1-70B-Instruct
  temperature: 0
  scaling_config:
    batch_size: 128
    concurrency: 3
    num_gpus: 2
    custom_resources:
      accelerator_type:H100: 1
num_mcq_questions: 5


In [ ]:
!anyscale job submit -f configs/summary_generation/mistral_finetuned_eval.yaml

In the logs for the above jobs, you should see the final path to the output summaries for both the models. 

Optionally, you can also obtain the summaries and scores for the `gpt-4o` model from OpenAI. Simply run:

In [ ]:
!anyscale job submit -f configs/summary_generation/gpt4o_eval.yaml

## Get Evaluation Statistics

We've provided a convenient script `src/scripts/get_eval_stats.py` to get evaluation statistics and obtain the "win rate" of the DPO model (the percentage of times the DPO model performs better than the baseline). We've provided an example configuration below. 

In [ ]:
# make sure to substitute -outputs-path with your path
!python src/scripts/get_eval_stats.py --outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_dpo_model/test/ --baseline-outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/test/  

# (Optional): if you obtained results for GPT-4o, you should uncomment and run the following command instead
# !python src/scripts/get_eval_stats.py --outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_dpo_model/test/ --baseline-outputs-path s3://air-example-data/preference-tuning-summarization-example/summary_generation_base/test/  --gpt4o-outputs-path <add-path-to-gpt4o-results>

You should see the following results:

```text 
╒═════════════════════════╤═══════════╤════════════╤═══════════╕
│         Metric          │   Model   │  Baseline  │  GPT-4o   │
╞═════════════════════════╪═══════════╪════════════╪═══════════╡
│      Accuracy >=3       │ 65.4286 % │ 43.0476 %  │ 37.2381 % │
├─────────────────────────┼───────────┼────────────┼───────────┤
│      Accuracy >=4       │ 25.7143 % │ 13.5238 %  │ 10.0000 % │
├─────────────────────────┼───────────┼────────────┼───────────┤
│   Median Compression    │ 11.5794 % │ 12.7316 %  │ 8.0496 %  │
├─────────────────────────┼───────────┼────────────┼───────────┤
│    Mean Compression     │ 13.0029 % │ 14.3444 %  │ 9.3554 %  │
├─────────────────────────┼───────────┼────────────┼───────────┤
│   Failed Compressions   │ 0.0000 %  │  0.0000 %  │ 0.0000 %  │
├─────────────────────────┼───────────┼────────────┼───────────┤
│ Contains OOD Characters │ 0.0000 %  │  0.0952 %  │ 0.0000 %  │
╘═════════════════════════╧═══════════╧════════════╧═══════════╛


Model Win Rate against Baseline: 74.0000 %
GPT-4o Win Rate against Baseline: 64.8095 %
```

Our fine-tuned model is able to generate much better summaries, that are more concise (compression ratio is lower) with lesser out-of-distribution characters (gibberish tokens) than the baseline. You can see more details on the same in our blog!

## Summary

Congrats! You have now fine-tuned an open source model on preference data. As a quick recap, here's what we demonstrated in this notebook:
1. Synthetically generating preference data for DPO 
2. DPO fine-tuning of a language model on the Anyscale Platform
4. Evaluating the model against the baseline and GPT-4o, and analysing the results.